### Look at how many trials per condition we have for confidence ccgp, after balancing

### Result: 
- filtering then labeling does make a difference, though not a big one... 
- probably the drop in decoding accuracy is due to the amount of trials lost by fi

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.pseudo_classifier_utils as pseudo_classifier_utils
import utils.classifier_utils as classifier_utils

import utils.io_utils as io_utils

import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import matplotlib
import utils.spike_utils as spike_utils
import utils.subspace_utils as subspace_utils
from trial_splitters.condition_trial_splitter import ConditionTrialSplitter 
from utils.session_data import SessionData
from constants.behavioral_constants import *


import scipy


In [ ]:
# the output directory to store the data
OUTPUT_DIR = "/data/res/pseudo"
# path to a dataframe of sessions to analyze
# SESSIONS_PATH = "/data/patrick_scratch/multi_sess/valid_sessions.pickle"
SESSIONS_PATH = "/data/valid_sessions_rpe.pickle"

# path for each session, specifying behavior
# path for each session, for spikes that have been pre-aligned to event time and binned. 
SESS_SPIKES_PATH = "/data/firing_rates/{sess_name}_firing_rates_{pre_interval}_{event}_{post_interval}_{interval_size}_bins_1_smooth.pickle"


DATA_MODE = "FiringRate"
EVENT = "StimOnset"  # event in behavior to align on
PRE_INTERVAL = 1000   # time in ms before event
POST_INTERVAL = 1000  # time in ms after event
INTERVAL_SIZE = 100  # size of interval in ms

In [12]:
def min_trials_per_cond(row, filter_then_label=True):
    sess_name = row.session_name

    behavior_path = SESS_BEHAVIOR_PATH.format(sess_name=sess_name)
    beh = pd.read_csv(behavior_path)
    valid_beh = behavioral_utils.get_valid_trials(beh)
    feature_selections = behavioral_utils.get_selection_features(valid_beh)
    valid_beh = pd.merge(valid_beh, feature_selections, on="TrialNumber", how="inner")
    beh = behavioral_utils.get_feature_values_per_session(sess_name, valid_beh)
    beh = behavioral_utils.get_max_feature_value(beh)
    beh = behavioral_utils.calc_feature_probs(beh)
    beh = behavioral_utils.calc_feature_value_entropy(beh)

    no_filt_beh = behavioral_utils.calc_confidence(beh, num_bins=2, quantize_bins=True)
    no_filt_num = behavioral_utils.get_min_num_trials_by_condition(no_filt_beh, ["MaxFeatDim", "ConfidenceBin"])

    # filter by max chosen, also by dimension of interest
    filter_first_beh = behavioral_utils.filter_max_feat_chosen(beh)
    filter_first_beh = behavioral_utils.calc_confidence(filter_first_beh, num_bins=2, quantize_bins=True)
    filter_first_num = behavioral_utils.get_min_num_trials_by_condition(filter_first_beh, ["MaxFeatDim", "ConfidenceBin"])

    label_first_beh = behavioral_utils.calc_confidence(beh, num_bins=2, quantize_bins=True)
    label_first_beh = behavioral_utils.filter_max_feat_chosen(label_first_beh)
    label_first_num = behavioral_utils.get_min_num_trials_by_condition(label_first_beh, ["MaxFeatDim", "ConfidenceBin"])

    return pd.Series({
        "session": sess_name, 
        "filter_then_label_num": filter_first_num, 
        "label_then_filter_num": label_first_num,
        "no_filt_num": no_filt_num
    })


In [13]:
valid_sess = pd.read_pickle(SESSIONS_PATH)
res = valid_sess.apply(min_trials_per_cond, axis=1)

In [14]:
res

,session,filter_then_label_num,label_then_filter_num,no_filt_num
0,20180709,49,51,72
5,20180920,85,73,147
7,20180918,52,43,93
8,20180912,58,50,95
10,20181008,61,47,96
11,20181002,65,66,103
15,20180806,49,52,74
17,20180801,73,54,115
22,20181005,78,64,124
27,201807250001,7,8,9


In [8]:
res.filter_then_label_num.sum() - res.label_then_filter_num.sum()

258